In [20]:
from landlordai.game.player import LearningPlayer
from landlordai.sim.simulate import Simulator
from landlordai.sim.game_stats import GameStats

import numpy as np
from tqdm import tqdm
import pickle
import random

import pkg_resources
pkg_resources.get_distribution('landlord-ai').version

'0.1.32'

In [22]:
!rm -r models
!gsutil -m cp -r gs://landlord_ai/models .

/snap/google-cloud-sdk/126/lib/third_party/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
Copying gs://landlord_ai/models/4_10_actualq1_model0/history.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model0/position.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model1/history.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model1/position.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model10/history.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model10/position.h5...            
Copying gs://landlord_ai/models/4_10_actualq1_model14/history.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model15/history.h5...             
Copying gs://landlord_ai/models/4_10_actualq1_model14/position.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model15/position.h5...            
Copying gs://landlord_ai/models/4_10_actualq1_model16/history.h5...        

Copying gs://landlord_ai/models/4_11_actualq2_model1/history.h5...              
Copying gs://landlord_ai/models/4_11_actualq2_model1/position.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model10/history.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model10/position.h5...            
Copying gs://landlord_ai/models/4_11_actualq2_model11/position.h5...            
Copying gs://landlord_ai/models/4_11_actualq2_model11/history.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model12/history.h5...
Copying gs://landlord_ai/models/4_11_actualq2_model20/position.h5...            
Copying gs://landlord_ai/models/4_11_actualq2_model20/history.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model3/history.h5...              
Copying gs://landlord_ai/models/4_11_actualq2_model2/position.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model21/history.h5...             
Copying gs://landlord_ai/models/4_11_actu

Copying gs://landlord_ai/models/4_9_actualq3_model23/position.h5...             
Copying gs://landlord_ai/models/4_9_actualq3_model30/history.h5...              
Copying gs://landlord_ai/models/4_9_actualq3_model24/position.h5...
Copying gs://landlord_ai/models/4_9_actualq3_model3/position.h5...              
Copying gs://landlord_ai/models/4_9_actualq3_model3/history.h5...               
Copying gs://landlord_ai/models/4_9_actualq3_model30/position.h5...             
Copying gs://landlord_ai/models/4_9_actualq3_model31/history.h5...              
Copying gs://landlord_ai/models/4_9_actualq3_model32/history.h5...              
Copying gs://landlord_ai/models/4_9_actualq3_model32/position.h5...             
Copying gs://landlord_ai/models/4_9_actualq3_model31/position.h5...             
Copying gs://landlord_ai/models/4_9_actualq3_model33/position.h5...             
Copying gs://landlord_ai/models/4_9_actualq3_model34/position.h5...             
Copying gs://landlord_ai/models/4_9_actua

In [23]:
models_dir = 'models/'
models = ['4_11_actualq2_model11', '4_7_actualq5_model10', '4_9_actualq1_model1', '4_9_actualq1_model21', '4_9_actualq2_model21',
'4_9_actualq4_model21', '4_10_actualq1_model4', '4_10_actualq2_model1', '4_10_actualq2_model12'] + ['4_2_sim4_model15']
        
#models = ['3_29_sim10_model0', '3_29_sim11_model0'] + ['3_29_sim12_model0'] + \
#            ['3_29_sim13_model' + str(i) for i in range(1, 5)] + ['3_30_sim1_model' + str(i) for i in range(3)]

models

['4_11_actualq2_model11',
 '4_7_actualq5_model10',
 '4_9_actualq1_model1',
 '4_9_actualq1_model21',
 '4_9_actualq2_model21',
 '4_9_actualq4_model21',
 '4_10_actualq1_model4',
 '4_10_actualq2_model1',
 '4_10_actualq2_model12',
 '4_2_sim4_model15']

In [24]:
player_pool = []

for i in range(1):
    player_pool.append(LearningPlayer(name='random', epsilon=0))


for model in models:
    player_pool.append(LearningPlayer(name=model, 
                                      epsilon=0,
                                      net_dir=models_dir + model,
                                      discount_factor=1,
                                      estimation_mode=LearningPlayer.NO_ESTIMATION))

player_pool_bc = sc.broadcast(player_pool)

In [25]:
cores_per_executor = 2

# since each spark executor fights for cpu space if we run a bunch of threads
def limit_threads(num_threads=2):
    import tensorflow as tf
    from tensorflow.python.eager import context

    _ = tf.Variable([1])

    context._context = None
    context._create_context()

    tf.config.threading.set_inter_op_parallelism_threads(num_threads)
    tf.config.threading.set_intra_op_parallelism_threads(num_threads)

In [26]:
num_games_per = 5000
parallel_core = sc.parallelize(zip(range(num_games_per), range(num_games_per))).repartition(2).cache()
_ = parallel_core.collect()

In [ ]:
import subprocess

def run_sim(x):
    # ensure proper random seeding
    import numpy as np
    import random
    random.seed(x)
    np.random.seed(x)
    
    # set the number of threads once per machine
    import tensorflow as tf
    current_threads = tf.config.threading.get_inter_op_parallelism_threads()
    if current_threads != cores_per_executor:
        limit_threads(cores_per_executor)
    
    # simulate games
    from landlordai.sim.simulate import Simulator
    
    sim = Simulator(10, player_pool_bc.value, competitor_pool = [], record_loser_pct=1.0)
    sim.play_rounds(debug=False)
    return sim.get_results()

def reduce_data(x, y):
    result_pairs_left = x
    result_pairs_right = y

    game_results_merged = result_pairs_left + result_pairs_right
    return game_results_merged
    

game_results = parallel_core \
                .partitionBy(num_games_per, lambda x: x) \
                .map(lambda x : run_sim(x[0] + (i * num_games_per))).reduce(reduce_data)



In [21]:
game_results[:10]

[(('4_7_actualq5_model10', '4_9_actualq1_model21'), ('random',)),
 (('4_7_actualq5_model10',), ('4_2_sim4_model14', '4_7_actualq6_model23')),
 (('4_7_actualq6_model23',), ('4_9_actualq1_model21', '4_2_sim4_model14')),
 (('4_9_actualq1_model21',), ('4_9_actualq1_model1', '4_7_actualq6_model23')),
 (('4_7_actualq5_model10', '4_2_sim4_model14'), ('4_9_actualq2_model21',)),
 (('4_9_actualq2_model21',), ('4_9_actualq1_model21', '4_2_sim4_model14')),
 (('4_2_sim4_model15', '4_7_actualq5_model10'), ('4_7_actualq6_model23',)),
 (('4_2_sim4_model14',), ('4_9_actualq1_model1', '4_7_actualq5_model10')),
 (('4_9_actualq1_model1', '4_9_actualq2_model21'), ('4_9_actualq1_model21',)),
 (('4_2_sim4_model15', '4_9_actualq1_model21'), ('4_9_actualq2_model21',))]

In [30]:
random.shuffle(game_results)
stats = GameStats(player_pool_bc.value, game_results)
stats.print_player_stats()

Player Stats:

1597.7819533574705 0.5331250673273726 4_9_actualq1_model21
1567.701478156392 0.5378628987746992 4_9_actualq4_model21
1565.5985506293766 0.5208690987124464 4_10_actualq2_model1
1542.2671856790178 0.522326174782054 4_10_actualq1_model4
1538.8600366273395 0.5286755386565273 4_9_actualq2_model21
1529.8769918438397 0.5300519551544982 4_11_actualq2_model11
1527.4582172008386 0.5382212723984876 4_10_actualq2_model12
1523.1465274779375 0.5204539313699 4_9_actualq1_model1
1504.8840711664282 0.5225042662116041 4_7_actualq5_model10
1487.672369718523 0.5190804061150659 4_2_sim4_model15
1146.845866494185 0.1857616101006313 random
